# Catalog ICRS to observed (detailed)

In this example we will show step by step calculations to convert ICRS catalog coordinates of a star to its observed coordinates. We follow the steps shown by *Patrick Wallace* in this [PDF file](https://syrte.obspm.fr/iau/iauWGnfa/ExPW04.pdf).

The observation site is located at 9◦.712156 E, 52◦.385639 N, 200m above sea level. The time of observation is UTC 2003/08/26 00:37:38.973810.

The target of observation is a fictitious Tycho 2 star, epoch 2000:
- [ α, δ ] = 353◦.22987757, +52◦.27730247
- Proper motions: μα cos δ = +22.9mas/year, μδ = −2.1mas/year
- Parallax 23mas
- Radial velocity +25 km/s

The Earth orientation parameters (from IERS):
- DUT1: −0.349535 s
- δX, δY corrections: +0.038, −0.118 (mas)
- polar motion xp, yp = 0.259371, 0.415573 (arcsec)

Let's begin by importing libraries


In [1]:
import erfa
from iers import EOP
import numpy as np

## Step 1: prepare

In the first step, we should assigning the given data to variables. The star data are: ICRS coordinates, proper motion, parallax and radial velocity. You have to pay attetion to units. 

The Right Ascension (RA) and Declination (DEC) are usually expressed in *degrees* in astronomical catalogues, but, when using in SOFA functions, as `rc` and `dc`, we should convert them to *radians*. We can do this by multiplying by `DD2R`.

The unit used for pmRA (RA component of proper motion) and pmDEC (DEC component of proper motion) in many catalogues is *milliarcsec/year*, but in SOFA functions we should convert them to *radians/year*. We can convert *milliarcsec/year* to *radians/year* by multiplying by `DMAS2R`. There's something else you should be aware. The values indicated as "pmRA" in many catalogues, such as Hipparcos, are in fact pmRA multiplied by cosine of DEC. These values are NOT time derivatives of RA. In SOFA functions, `pr` and `pd` are pmRA and pmDEC in *radians/year*. To prepare `pr`, the cosine of DEC should be removed as follows:

`pr = np.arctan2(pmRA * erfa.DMAS2R, np.cos(dc))`

Parallax in many catalogues is expressed as *milliarcsec*, but if SOFA, `px`  should be in *arcsec*. The unit of radial velocity is *km/s* and it is indicated with `rv`.

In [2]:
# Star data
rc = 353.22987757 * erfa.DD2R
dc = 52.27730247  * erfa.DD2R
pr = np.arctan2(22.9 * erfa.DMAS2R, np.cos(dc))
pd = -2.1 * erfa.DMAS2R
px = 23.0 * 0.001
rv = 25.0

The time of observation is in UTC. We should convert it to Terrestrial Time (TT) because most of the SOFA functions need *TT scale*. The *time format* used by SOFA is *Julian Date*. To increase precision, SOFA uses two numbers to save time. This is called *2-part Julian Date* format. Sum of these two numbers gives the time in Julian Date. 

First, we should convert our UTC time to *2-part Julian Date* format, using `dtf2d` function. Then, we convert the results to *International Atomic Time (TAI)* scale, using `utctai` function. Finally, we use `taitt` function to convert TAI scale to TT scale.

In [3]:
# Time of observation (convert UTC to TT)
utc1, utc2 = erfa.dtf2d("UTC", 2003, 8, 26, 0, 37, 38.973810)
tai1, tai2 = erfa.utctai(utc1, utc2)
tt1, tt2 = erfa.taitt(tai1, tai2)

To get Earth Orientation Parameters, we use `EOP` class from the `iers` python package. This class has the method `get_eop` that accepts a time in UTC and returns a dictionary containing parameters. You can enter time as *Julian Date*, *datetime*, or time string in ISO format. Since we have already the UTC time in JUlian Date, we use it as follows:

In [4]:
eop = EOP().get_eop(utc1 + utc2)
print(eop)

{'px': 0.25937117787962316, 'py': 0.41557267392846753, 'ut1_utc': -0.34953485580035626, 'dx': 0.03767978379037231, 'dy': -0.11784986281348393}


The unit of `px` and `py` is *arcsec*; the unit of `ut1_utc` is *seconds*; and finally, the unit of `dx` and `dy` is *milliarcsec*. Let's put them in variables and convert them to units that are required by SOFA.

In [5]:
# Earth Orientation Parameters
dut1 = eop['ut1_utc']
xp = eop['px'] * erfa.DAS2R
yp = eop['py'] * erfa.DAS2R
dx = eop['dx'] * erfa.DMAS2R
dy = eop['dy'] * erfa.DMAS2R

In the following steps, we want to print the results. Since the results are in cartesian and radians, let's create a very simple function to convert them to spherical and degrees and print them to the output as well as a text message.

In [6]:
def printSph(p, text):
    w, d = erfa.c2s(p)
    r = erfa.anp(w)
    print(f"{text}: {r*erfa.DR2D}, {d*erfa.DR2D}")

## Step 2: Position-velocites (PVs) of the Earth

Before we start coordinates convertions, we need heliocentric and barycentric position-velocity vectors of the Earth. We can use the function `epv00` or get it from other resources (for example NASA JPL). The unit for the position vectors is *au* and the unit for the velocity vectors is *au / day*.

In [7]:
# Earth heliocentric and barycentric position-velocity vectors
pvh, pvb = erfa.epv00(tt1, tt2)

print('Position of Earth (heliocentric):', pvh[0])
print('Velocity of Earth (heliocentric):', pvh[1])
print('Position of Earth (barycentric) :', pvb[0])
print('Velocity of Earth (barycentric) :', pvb[1])

Position of Earth (heliocentric): [ 0.89530671 -0.43036218 -0.18658314]
Velocity of Earth (heliocentric): [0.00770924 0.01392773 0.00603814]
Position of Earth (barycentric) : [ 0.8981304  -0.4336632  -0.18805818]
Velocity of Earth (barycentric) : [0.00771448 0.01393305 0.00604026]


Now that we have PV vectors of the Earth, we should caculate four factors that will be necessary later. 

1. First, we should calculate the distance from Sun to the observer. This is simply the magnitude of the heliocentric position vector of the Earth. In our code, we call it `em`. 

2. We also need the unit vector for the direction from the Sun to the observer. This is the heliocentric position vector divided by the `em` value. We call this new vector `eh`.

3. We need the barycentric observer velocity. This is the barycentric velocity vector and we already have it, but we need it to be expressed in unit of speed of light. The value of the speed of light in *au / day* can be obtained from the constant `DC`. So, we divide the barycentric observer velocity by `DC`. We call the result `v`.

4. We use `v` to calculate the reciprocal of Lorenz factor and we call it `bm1`.

In [8]:
# distance from Sun to observer (au) & Sun to observer (unit vector)
em = np.sum(pvh[0]**2) ** 0.5
eh = pvh[0] / em

# barycentric observer velocity (vector, c)
v = pvb[1] / erfa.DC #Note: DC is Speed of light (au per day)

# reciprocal of Lorenz factor
bm1 = np.sqrt(1 - np.sum(v**2))

## Step 3: BCRS coordinates

Until here, we had nothing to do with the coordinates of the star. Now we start by applying proper motion and parallax to the catalog coordinates of the star, using pmpx. Note that this function take the time as julian years.

We need to find the proper motion time interval and convert it to Julian years.

The reference epoch (J2000.0) expressed as Julian Date is 2451545.0, so, we subtract it from the observation time and divide the results by 365.25. Note that there are ecactly 365.25 days in a Julian Year. Hence, `pmt` is the number of Julian years since J2000.

In [9]:
# PM time interval (Julian years)
pmt = ((tt1 - 2451545.0) + tt2) / 365.25

# Proper motion and parallax, giving BCRS coordinate direction
pco = erfa.pmpx(rc, dc, pr, pd, px, rv, pmt, pvb[0])

printSph(pco, "Astrometric (BCRS)")

Astrometric (BCRS): 353.2299188909816, 52.27730584234127


We apply the light deflection by the Sun, using `ldsun`.

In [10]:
# Light deflection by the Sun, giving BCRS natural direction
pnat = erfa.ldsun(pco, eh, em)
printSph(pnat, "With light deflection (BCRS)")

With light deflection (BCRS): 353.22991848170153, 52.277305175083455


## Step 4: GCRS coordinates

Applying the aberration, using ab, gives us the GCRS coordinates of the star.

In [11]:
# Aberration, giving GCRS proper direction
ppr = erfa.ab(pnat, v, em, bm1)
printSph(ppr, "Abberation (GCRS)")

Abberation (GCRS): 353.23789320692833, 52.276952625332505


## Step 5: CIRS coordinates

To cenvert GCRS to CIRS, we need *Bias-Precession-Nutation (BPN)* matrix. We can get this matrix by calling `pnm06a(tt1, tt2)`. But, here we want more control over the calculations: we want to apply IERS corrections (`dx` and `dy`). So, after getting the intial *BPN*, we get the CIP coordinates from this matrix by calling `bpn2xy`. Then, we apply the corrections (adding `dx` and `dy` to `x` and `y`). Now that we have the exact coordinates of the CIP (`x` and `y`), we can get the CIO locator `s` by calling the function `s06`. By passing `x`, `y` and `s` to `c2ixys` we get more precise *BPN* matrix.

In [12]:
#r = erfa.pnm06a(tt1, tt2)
#x, y = erfa.bpn2xy(r)
x, y = erfa.xy06(tt1, tt2)

x += dx # Apply IERS corrections
y += dy # Apply IERS corrections
s = erfa.s06(tt1, tt2, x, y)

# More precise BPN matrix
bpn = erfa.c2ixys(x, y, s)

print(bpn)

[[ 9.99999946e-01  9.55322624e-09 -3.29956841e-04]
 [-1.85693025e-08  1.00000000e+00 -2.73250137e-05]
 [ 3.29956841e-04  2.73250183e-05  9.99999945e-01]]


Now that we have the GCRS coordinates of the star, we can multiply the *BPN* matrix to it in order to get CIRS coordinates of the star.

In [13]:
# Bias-precession-nutation, giving CIRS proper direction
pi = erfa.rxp(bpn, ppr)
printSph(pi, "CIRS")

# CIRS RA,Dec
w, di = erfa.c2s(pi)
ri = erfa.anp(w)

CIRS: 353.2330020893926, 52.29554174098014


## Step 6: Observed coordinates

We have now the CIRS coordinates of the star (`ri`, `di`). The remaining part can be achieved by calling a single function `atio13`, which converts CIRS to observed. If your provide the atmospheric condition of the obseving site (applying refraction) you will get the observed coordinates. Thses variables are ambient pressure (`phpa`), temperature (`tc`), relative humidity (`rh`) and wavelength (`wl`). If you don't provide them by passing zero, you will get the topocentric coordinates. We will try both.

In [14]:
elong = 9.712156 * erfa.DD2R
phi = 52.385639  * erfa.DD2R
hm = 200.0
phpa = 1000.0 #Ambient pressure (HPa)
tc = 20.0     #Temperature (C)
rh = 0.70     #Relative humidity (frac)
wl = 0.55     #wavelength  (microns)

# CIRS to topocentric
(aot, zot, hot, dot, rot) = erfa.atio13(
    ri, di, utc1, utc2, dut1,
    elong, phi, hm, xp, yp,
    0.0, 0.0, 0.0, 0.0
)
print(f"Topocentric: {hot*erfa.DR2D}, {dot*erfa.DR2D}")
print(f"Topocentric: {aot*erfa.DR2D}, {90-zot*erfa.DR2D}")

# CIRS to observed
(aob, zob, hob, dob, rob) = erfa.atio13(
    ri, di, utc1, utc2, dut1,
    elong, phi, hm, xp, yp,
    phpa, tc, rh, wl
)
print(f"Observed   : {aob*erfa.DR2D}, {90-zob*erfa.DR2D}")

Topocentric: -0.29507962859885295, 52.29549062781548
Topocentric: 116.44983895116414, 89.79843387509257
Observed   : 116.44983895116414, 89.79848799947808
